In [3]:
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import os
import json
import shutil
import random

In [4]:
with open("train.json") as f:
    data = json.load(f)

In [9]:
data['v_' + 'lTFCUuLtUJs']

{'duration': 216.04,
 'timestamps': [[0, 76.69], [65.89, 152.31], [152.31, 206.32]],
 'sentences': ['A large group of people are seen standing around a track leading into several people running down the road.',
  ' A woman is seen speaking to the camera while several people are seen running and drinking water.',
  ' More people speaking to the camera while showing them run in different locations.']}

In [8]:
data['v_' + 'lTFCUuLtUJs']['sentences']

['A large group of people are seen standing around a track leading into several people running down the road.',
 ' A woman is seen speaking to the camera while several people are seen running and drinking water.',
 ' More people speaking to the camera while showing them run in different locations.']

In [34]:
pid = 'lTFCUuLtUJs'
cap = cv2.VideoCapture("/datasets/home/22/422/sramaswa/CSE251B/activityNetData/training/Running_a_marathon/lTFCUuLtUJs")
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
curr_data = data['v_' + pid]
duration = curr_data['duration']
timestamps = curr_data['timestamps']
sentences = curr_data['sentences']
for i, caption in enumerate(sentences):
    start = int(timestamps[i][0] / duration * num_frames)
    end = int(timestamps[i][1] / duration * num_frames)
    print(start, end, caption)

num_frames


0 2297 A large group of people are seen standing around a track leading into several people running down the road.
1973 4562  A woman is seen speaking to the camera while several people are seen running and drinking water.
4562 6180  More people speaking to the camera while showing them run in different locations.


6472

In [44]:
def build_data(directory, root_destination, metadata, dim, collected_frames):
    annotations = []
    for action in os.listdir(directory):
        print(action)
        fullpath = os.path.join(directory, action)
        destination_action = root_destination + "/" + action
        try:
            os.mkdir(destination_action) 
        except:
            shutil.rmtree(destination_action)
            os.mkdir(destination_action)
        for pid in os.listdir(fullpath):
            if 'v_{}'.format(pid) not in metadata:
                print("skipping {}, not found in metadata".format(pid))
                continue
            captions = metadata['v_{}'.format(pid)]['sentences']
            duration = metadata['v_{}'.format(pid)]['duration']
            timestamps = metadata['v_{}'.format(pid)]['timestamps']
            fullpath2 = os.path.join(fullpath, pid)
            cap = cv2.VideoCapture(fullpath2)
            num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for i, caption in enumerate(captions):
                start = int(timestamps[i][0] / duration * num_frames)
                end = int(timestamps[i][1] / duration * num_frames)
                print(num_frames, start, end, caption)
            
                destination_action_pid_i = destination_action + "/" + pid + "_" + str(i)
                try:
                    os.mkdir(destination_action_pid_i) 
                except:
                    shutil.rmtree(destination_action_pid_i) 
                    os.mkdir(destination_action_pid_i) 

                if (end - start) < collected_frames:
                    counter = 1
                    collected_frames_i = end - start
                else:
                    counter = int((end - start)/collected_frames)
                    collected_frames_i = collected_frames
                annotation = "{} 1 {} {}\n".format(os.path.join(action,pid + "_" + str(i)), str(collected_frames_i), caption)
                annotations.append(annotation)
                for j in range(0, collected_frames_i):
                    k = j * counter
                    cap.set(1, start + k)
                    ret, frame = cap.read()
                    if ret:
                        final_destination = "{}/img_{:05d}.jpg".format(destination_action_pid_i, j+1)
                        resized_frame = cv2.resize(frame,(dim,dim))
                        cv2.imwrite(final_destination, resized_frame)
                break
    f = open("{}_annotations.txt".format(root_destination),"w") 
    f.writelines(annotations)
    f.close()

In [42]:
# with open("train.json") as f:
#     metadata = json.load(f)
# directory = "/datasets/home/22/422/sramaswa/CSE251B/activityNetData/training"
# root_destination = "train"
# dim = 128
# frames_to_collect = 20
# build_data(directory, root_destination, metadata, dim, frames_to_collect)

Running_a_marathon
6472 0 2297 A large group of people are seen standing around a track leading into several people running down the road.
5444 0 163 A graphic with some keywords forming the number '2' is shown.
2023 0 2023 A man is running down a street.
4024 261 301 A man is crossing the finish line of a race.
2587 0 1823 Students are interviewed and then run a race.
2362 0 153 A news station ad appears on the screen.
2270 0 1634 An old woman talks before run in a marathon where there is a lot of people participate.
967 4 957 A man rides a bicycle behind three men who are running in a professional race on a paved freeway lined with onlookers, spectators and participants.
3886 0 310 We see a pink opening scene.
3534 0 141 An intro for a running video that says Running appears on screen.
Swimming
4375 0 481 A youth talks inside a home, then he gets outside the home.
5716 0 2257 A man is seen swimming in a pool holding various objects in his hands and showing how to put them on.
2148 0 

1124 0 157 We see people holding their phones up to record.
1081 0 16 A lady looks behind her with delight.
2987 0 2837 A man sits and plays a black grand piano in an empty auditorium.
1118 0 139 A man and a woman are standing by a piano.
271 0 271 A man in black is sitting and playing at a piano while a man in red is standing near drums.
4223 0 4223 a man and a woman are sitting in front of a piano playing it together.
Playing_violin
3457 34 3232 Four little girls , dressed in white dresses with red sashes, stand in a line, on a stage and play the violin together.
1984 0 327 The woman is sitting by the fireplace holding a violin.
5684 341 880 A man is shown flipping a violin in his hands, playing a popular song on his violin.
4125 61 2227 A man is shown playing a violin in front of a video game being played on tv.
3872 561 987 There's man dressed in a black sweater playing the violin in a church.
7115 960 2952 A violinist is playing the violin in a music room.
6149 0 1752 A man holdin

2045 0 163 A man and a woman dressed in snow clothing are walking in the snow and they begin to drill into the ice ground with hand held red crank long drills.
4768 0 643 A man is holding a fishing pole over an ice fishing hole.
6789 0 6789 A person is walking slow on ice,trying to to crack it.
3226 0 1500 A man is sitting on a bucket ice fishing.
Brushing_hair
1781 17 596 A woman is seen standing beside a girl sitting down brushing her hair.
216 23 178 A young girl wearing a gray sweater with long red colored hair is brushing her hair.
1247 0 212 A girl in a bra is standing in her bathroom and brushing her hair.
547 0 243 A woman stands in front of a mirror smiling and begins brushing her hair.
2222 0 2177 A teenage girl brushes her hair.
2588 25 2588 A baby is shown laying in bed while a comb gently brushes it's hair.
4752 94 641 A man stands in front of the camera, then starts combing his hair forward.
5316 0 956 A person with crazy hair appears and leads into her on a space ship wi

6214 528 838 The video is a tutorial on how to make Alfredo sauce pasta.
5191 0 5191 A woman is standing in a kitchen.
2507 12 100 The credits of the clip are shown.
4026 0 4026 This video shows you how to make spaghetti with sausage with a simple tomato sauce.
5058 758 3287 A woman is adding ingredients to a pan on the stove.
3709 0 3709 A woman is in a kitchen making a tomato sauce in the kitchen.
Grooming_horse
2422 0 545 An intro leads into various pictures of horse tools and a woman speaking to the camera holding a brush.
2562 0 1767 woman is standing next to a horse holding a brush combing the horse's hair.
1834 0 192 A woman is in a stable with a white horse.
2058 0 854 A woman grooms the right body of a horse, she brush the neck,body and legs.
917 13 238 A close up of a horse is seen standing in a fenced in area.
1402 0 434 A small horse is outside in a field  being held by a rope that is attached to a belt wrapped around the horse's neck.
1072 0 380 A black horse is standing i

3362 50 588 A man in yellow shirt and black tights, walked towards the bike, bend over and unscrew the screw.
678 0 417 A man replaces a tire on his bike rim using tools.
4580 0 1740 A young man pulls a wheel out and shows the proper structure of a wheel.
5461 0 2157 A person is seen standing in front of a picnic table speaking to the camera while holding a bike wheel.
6096 30 853 A person is talking on camera while showing various tools on the ground.
2264 0 260 A man stops his bike on the sidewalk.
626 0 626 man wearing a green shirt is assembling a bicycle on the middle of a living room.
4835 894 4835 A person repairs a bike chain and installs the chain on a bike interspersed with an image of a woman talking.
Mixing_drinks
5650 0 197 An intro is shown with a drink being poured into a glass and a caption about drink mixing.
1596 135 446 A bartender talks about how to make a proper cocktail.
2038 0 112 A man stands behind a bar talking to the camera.
1902 142 323 Maria is demonstratin

...

Kayaking
2860 14 915 A man is seen sitting in a kayak while speaking to the camera and holding a paddle.
4368 305 764 People are paddling in kayaks in a body of water.
5779 0 2947 A person's arms are seen moving back and fourth in the water controlling a kayak under some tombs sticking out of the water.
283 32 146 A person wearing a blue jacket and orange helmet is on an overturned raft in the water.
1288 0 508 A kayak rider drifts in the bay of an ocean.
695 31 361 A man is sitting on a rock by a fast flowing river.
3479 0 260 A person on a skateboard moves over and ramp and up on a corner.
245 0 112 A man sitting in a kayak pushing himself around swirling water using his paddles for support.
1672 0 777 A person is seen climbing into the back of a kayak with another man and paddling alone the water.
694 0 680 A kayak rider paddles against the current and maneuvers back and forth with his hips and paddle.
5364 0 241 A man stands  outside a tent.
4635 0 4426 An athletic man is swimming 

2113 0 665 A groip of dancers are on stage doing a routine for the audience.
3635 0 3307 A man is standing on a stage performing different moves.
2140 0 374 A young man in white runs on the stage dancing.
4795 0 4771 Hiphop dancers perform in front of a crowd.
4313 0 2997 A man stands in a dance studio and shows how he sets up for different dance postures.
6298 0 6234 Three individuals sit on stage and have a conversation.
3751 0 243 A young man in blue t-shirt walks through small dance studio room.
1950 0 126 A young male dressed in an red jumpsuit is on the stage break dancing.
7062 176 3248 A man is seen laying on the floor and begins demonstrating how to properly perform dance moves as well as the wrong way to perform them.
3914 0 39 The scene comes into focus showing the outside of a building, with some text at the bottom that says it is a school for breakdancing.
1860 0 148 The screen opens up with an ACDC promo.
5282 0 1452 A man explains while stands on a fenced court.
5500 0 2

2569 38 2054 A man and a woman perform various dance moves back and fourth in an empty room.
4049 0 668 A woman starts dancing next to a man in front of a gathered crowd.
1277 0 287 A girl dressed as a tomboy and a boy dressed in a red shirt are standing in a room across from each other in a messy room.
2479 24 2429 A man and a woman are performing a couple's dance routine while other couples dance around them.
3746 0 430 A woman wearing pajamas is talking and standing on a star decorated stage.
Tai_chi
3001 0 3001 man is wearing a black kimono and isdoing yoga movements.
2180 0 239 A man demonstrates Tai Chi combat with other person.
1803 0 1803 A man shows up on the screen wearing a martial arts uniform showing people how to do a side kick.
2659 26 624 A man performs martial arts fighting with people in a film, then a big door close and a woman helps the man fight.
2354 0 317 An intro is shown followed by two people and a man talking to the camera.
5073 0 963 A pregnant woman is slow

4937 469 938 There's a woman in a pink dress sitting with a man in a gray shirt.
2862 0 772 A little girl wearing a brown bow is standing in front of the sink brushing her teeth.
2504 0 250 We see a boy brushing his teeth and dancing in the bathroom.
1199 0 83 A lady holds a toothbrush as a child sits on a counter.
5717 543 1915 A girl wearing a red shirt and white shorts is standing in her bathroom at the sink with a toothbrush in her hand.
198 0 198 A woman is brushing her teeth in a bathroom.
3296 49 2982 A video tutorial plays with a woman talking in the background about how to brush your teeth.
926 0 50 A girl in a yellow shirt is brushing her teeth in a bathroom.
3837 0 1438 man is sitting in a handwash and another man enters in the bathrrom and start a conversation.
409 0 100 A young man is seen looking into the camera holding a toothbrush.
Playing_lacrosse
2828 84 395 Two men talk to the camera, interspersed with action footage of them playing lacrosse.
4088 0 122 A man is show

1056 0 1056 A group of skiers are riding down a hill.
4395 0 615 A man is skiing very fast down a hillside and hops one for fun.
4838 0 193 People are standing on a hill of snow.
2930 0 600 A person is shown drawing several lines on a piece of paper that leads into several fast motion clips of snowy areas.
5566 0 5566 This video is a tutorial showing people how to ski correctly.
3343 217 718 A man in a blue coat is standing in the snow talking.
3970 0 1846 A large group of people are sitting around a table and transitions into people sitting on a ski lift.
4559 0 638 People are skiing in doors in between each other.
Sailing
3101 248 325 Ships are shown in the water.
4234 0 2328 A humongous red, black and white ship is shown on the water, then a smaller ship is shown quickly approaching the boat that is filming and suddenly it crashes into the boat and it has black smoke coming from the top of it.
5617 0 1881 Several shots are shown of people pushing sail boats into the water as well as

In [45]:
# with open("val_1.json") as f:
#     metadata = json.load(f)
# directory = "/datasets/home/22/422/sramaswa/CSE251B/activityNetData/validation"
# root_destination = "val"
# dim = 128
# frames_to_collect = 20
# build_data(directory, root_destination, metadata, dim, frames_to_collect)

Removing_ice_from_car
1600 232 1559 A man scrapes snow off of a blue car and then takes some of that snow, makes a snowball, and throws it in the direction of the camera, at an unseen person, until a hand holding a cigarette appears from behind the camera.
5800 0 2203 A snowy scene is shown, and a man walks through the snow of a parking lot to his car.
1227 0 1227 a man is holding a litle shovel and is cleaning the windows of a black car.
1778 0 1778 We see a person scraping ice off their windshield with the camera inside the car.
2285 0 274 A man opens an icy car door and snow falls off.
skipping UBQfURrVB_Y, not found in metadata
2188 0 744 We see a talking lady near her car on a snowy day.
Making_a_cake
4325 0 1513 woman is slicing a chocolate cake and its decorating it, put a second floor and some pink fondam.
3610 270 1101 A person puts flower, sugar, water and two eggs in a bowl.
3137 0 831 A man is seen writing on a piece of paper and shows a pan out of a truck.
6421 0 160 We se

3783 0 1097 A large auditorium is seen with a young dancer walking out on stage.
3437 34 1082 Various shots of a man are seen that lead into him performing with a baton.
5286 79 1242 A large group of people are seen standing around a city center waiting for people to arrive.
3164 63 3068 A woman is seen performing a routine on a large gymnasium floor with a baton in her hands.
2585 0 2585 Two girls are dancing with a baton.
408 2 405 A group of girls are in a gym practicing a baton twirling exercise.
Paintball
4320 21 1555 Soldiers holds firearms in a training camp wearing camouflage clothes, then, the men run to fight in a field shooting while their firearms.
5191 0 1816 a cabin in the woods is shown and a people wearing camouflage suits practicing shots to the cabin and hiding behind trees.
5572 0 2563 A camera pans all around a paintball course and shows a person aiming off into the distance.
7101 674 7101 A group of people play a game of paintball in an outdoor setting with props a

1761 0 378 Two men are shown in playing racket ball.
6790 0 6790 A man in an orange shirt is playing wall ball.
6189 0 2816 Two people are seen holding tennis rackets and begin hitting a ball around the court back and fourth to one another.
812 0 803 Three people play racquetball really badly.
Hammer_throw
1010 0 989 two men wearing purple shirts are sitting behind a net where a man will do a throw.
1627 0 113 A person is standing, preparing to throw the hammer in track and field on a the field in a large stadium.
1681 0 563 A man takes a ball attached to a rope and swings it around his body.
444 0 99 A lady stands holding a string with a ball.
829 12 443 An athlete is seen stepping up to a circle with people watching on the sides and her spinning around.
4654 0 302 man wearing a red shirt is making a shot put on a large green field.
1354 0 568 A young woman is seen standing behind a netting area with a swing in her hand and moving in slow motion.
Playing_badminton
2242 0 235 Two peopl

78 0 15 We see a man kick a ball and start running.
1182 0 171 A man runs to kick a kickball.
Roof_shingle_removal
1783 0 285 A picture of a red arrow is shown.
3921 58 3764 A man stands on a ladder next to a roof and talks to the camera while prodding at roof tiles with a metal tool.
1122 95 555 Some windows of a house are covered with planks.
1901 0 85 A large moldy hole is seen on the roof of a house.
2796 0 2796 A man is sitting on top of a roof.
156 0 64 A person is using a pitchfork to remove the roof materials.
3559 0 3345 The arm of a backhoe in shovel configuration peels the tiles off a roof.
6022 0 1023 A man is seen standing on a roof with a rope attached to him.
Playing_pool
1125 0 101 Two men are talking outside a building.
4558 0 4239 People are playing pool in a bar.
555 0 133 A close up is shown of a pool table with a man standing around the side.
3384 0 253 A man leans over a pool table, cue in hand.
3506 0 823 An intro leads into two men holding pool sticks and one hi

2885 0 1601 Various teams of people are seen standing around a field and leads into a man giving a speech in front of a large crowd.
4684 0 4496 Different groups compete in games of tug of war.
Windsurfing
5880 0 5880 A man is standing on a kite surf board.
330 0 143 A man is parasailing on the ocean.
2501 0 200 The road is blurred as a car speeds down it.
4279 0 106 A man on a board and a sail is in the ocean and words appear on the screen and say "Freestyle Practice Bonaire 2007".
5926 0 2044 A man sails on a sailboard on front the mountains.
307 0 307 A person is windsurfing in the body of water.
5458 190 5376 Two girls are sailing on a lake.
Ballet
4077 81 1895 A large group of people are seen moving out onto a large stage and performing a dance routine with one another.
4575 45 1303 A group of girls are seen standing at a ballet barre performing movements behind one another.
843 105 716 A woman and a man perform a ballet routine in a dance studio.
3387 0 3387 We see ballerinas on 

...

6818 0 1295 A group dressed in military clothing is outside.
6532 0 6532 Drummers are standing in a line drumming.
1980 0 534 A group of people are outside holding umbrellas watching the marching band perform in the parade.
3252 0 1186 A large group of people are seen walking around a street with a group of people dancing in the middle.
2278 0 2277 A marching band stands in formation outside of a historic building and with a set of drummers playing their instruments in stiff motions.
944 0 924 A marching band goes down the road in parade style.
Drinking_coffee
3347 0 702 An middle aged man is sitting in front of his desk talking.
1769 0 1653 A cat is dipping its paw into a mug and licking it off.
Grooming_horse
1655 0 422 A woman washes a horse's butt.
2763 0 1063 A woman is seen speaking to the camera while holding up tools and another woman standing behind her brushing a horse.
2869 0 487 A woman with blonde hair is standing outside of a stable next to a black horse.
1951 0 117 A blo

2526 0 265 A man is shown running that leads into an intro, followed by a photograph of him.
4786 0 2728 people are running in a field playing lacrosse.
6626 0 331 We see an opening screen with an image.
2850 0 2850 Two teams of girls are on a field playing field hockey.
5624 0 112 We see an opening title screen.
Sumo
5798 0 318 men are wearing together a costume and a man is helping them.
2885 0 807 big mena re standing on a ring getting ready to wrest.
2396 0 1593 Sumo wrestling demonstration is completed in front of fans.
6401 0 1728 A small television screen is on the wall and the person zooms in on the video.
5405 135 1837 Two men are seen kneeling down in front of one another and leads into a wrestling match between the two.
4990 0 1222 We we boys in diaper sumo gear running around in a gym, the boys then begin to wrestle.
Playing_piano
6990 0 139 A man is seated at a piano.
5955 59 4555 A man is seen playing a piano in the middle of a busy mall while people stand around him and 

2200 0 2200 A man is sitting on a riding lawn mower.
1180 0 407 A skinny woman is in her backyard cutting her yard with a push lawn mower and she is struggling.
4512 0 4512 The young man with headphones is pushing the grass cutter, while the man in gray sweater is guiding him.
5423 0 1491 A man is seen riding into frame on a lawn mower and looking at the camera.
Baking_cookies
3618 0 379 Butter is blended in a bowl.
4348 0 152 A whisk and several fruits are shown on a counter before the video actually starts.
5758 0 259 A tabletop with food ingredients appear on a wooden slat surface and fancy white wording appears in a special effect way on the left of the screen and it reads "Home Cooking adventure" and the website is displayed at the bottom left.
3815 0 419 Three black and white chocolate crinkles are places on a small blue saucer.
Tumbling
2635 0 2490 Girls are doing gymnastics tumbles on a blue mat.
3998 0 1139 An intro leads into several shots of various men performing martial ar

1911 0 133 A field of purple flowers is shown in an image.
Rollerblading
4022 0 4022 woman is on roller skate in a competition doing tricks around cones.
6956 104 3721 A camera pans around various people riding around on roller blades inside a mall.
2126 21 1243 An intro leads into a roller blading grinding down a rail as well as performing tricks in a skate park.
1014 20 263 A large group of people are seen standing around a city center and a man seen standing in the middle wearing roller blades.
Scuba_diving
5574 0 752 An intro leads into several people on a boat and one speaking to another.
5641 0 1889 A person is seen sitting at the top of the water and moving themselves slowly to the bottom.
5358 0 1768 Various bubbles are seen under water followed by a scuba diver swimming around and fish moving under the water.
4511 180 2571 A camera pans around a large ocean floor with fish swimming around and scuba divers seen.
1598 0 351 A leaf is shown floating through the water and two scub

Doing_kickboxing
1907 0 1840 A boy practices boxing with a kneeling adult.
3250 227 3103 This video shows viewers how to do a kickboxing elbow strike.
2822 0 578 Five young boys and a girl are in a large gym doing various exercises,such as push ups,jumping up and down,crunches and running through ladders on the floor.
1133 0 294 Two men are in a ring, kicking and blocking.
1284 12 327 A man and woman are seen moving around a boxing ring hitting each other with gloves.
Peeling_potatoes
558 0 558 This young black woman is peeling the onion over the garbage can in the kitchen.
1001 0 205 A woman wearing a bandanna and a grey shirt is sitting down on a grey couch carving something.
2244 22 751 A young girl is seen standing before a counter peeling potatoes with a peeler.
586 0 278 A young girl is seen peeling potatoes while looking up to the camera and other children moving around her.
Futsal
6714 0 6714 Teams play a game of indoor soccer.
5573 585 668 A crowd of people are watching a game

In [46]:
# Using readlines()
file1 = open('train_annotations.txt', 'r')
Lines = file1.readlines()

captions = set()
# # Strips the newline character
for line in Lines:
    caption = " ".join(line.strip().split()[3:])
    captions.add(caption)
print(len(captions))

file2 = open('val_annotations.txt', 'r')
Lines = file2.readlines()

# # Strips the newline character
for line in Lines:
    caption = " ".join(line.strip().split()[3:])
    captions.add(caption)
print(len(captions))

2578
3871


In [53]:
print(list(captions)[:10])

['A video into leads into several shots of bowlers throwing a ball down a lane and hitting pins.', 'Two men are seen playing a game of ping pong with each other hitting the ball back and fourth.', 'Water sprinklers shoot up out of the ground.', 'A guy on a motorcycle followed by other guys on motorcycles pull into a area while the first guy began to remove his helmet.', 'A person in a boat throws a board to someone in the water.', 'A team is on a field as a runner throws the ball, which the batter hits and is caught.', 'We see an opening screen with an image.', 'The camera is showing an arc welder in a shop.', 'A man stands on a ladder next to a roof and talks to the camera while prodding at roof tiles with a metal tool.', 'A man bends down and picks up a large weight.']


In [54]:
# build vocab
import vocab

In [55]:
activity_vocab = vocab.load_vocab(captions, 0)

Saved the vocab.


In [57]:
activity_vocab.word2idx

{'<pad>': 0,
 '<start>': 1,
 '<end>': 2,
 '<unk>': 3,
 'a': 4,
 'video': 5,
 'into': 6,
 'leads': 7,
 'several': 8,
 'shots': 9,
 'of': 10,
 'bowlers': 11,
 'throwing': 12,
 'ball': 13,
 'down': 14,
 'lane': 15,
 'and': 16,
 'hitting': 17,
 'pins': 18,
 '.': 19,
 'two': 20,
 'men': 21,
 'are': 22,
 'seen': 23,
 'playing': 24,
 'game': 25,
 'ping': 26,
 'pong': 27,
 'with': 28,
 'each': 29,
 'other': 30,
 'the': 31,
 'back': 32,
 'fourth': 33,
 'water': 34,
 'sprinklers': 35,
 'shoot': 36,
 'up': 37,
 'out': 38,
 'ground': 39,
 'guy': 40,
 'on': 41,
 'motorcycle': 42,
 'followed': 43,
 'by': 44,
 'guys': 45,
 'motorcycles': 46,
 'pull': 47,
 'area': 48,
 'while': 49,
 'first': 50,
 'began': 51,
 'to': 52,
 'remove': 53,
 'his': 54,
 'helmet': 55,
 'person': 56,
 'in': 57,
 'boat': 58,
 'throws': 59,
 'board': 60,
 'someone': 61,
 'team': 62,
 'is': 63,
 'field': 64,
 'as': 65,
 'runner': 66,
 ',': 67,
 'which': 68,
 'batter': 69,
 'hits': 70,
 'caught': 71,
 'we': 72,
 'see': 73,
 'an':